In [1]:
from src.features.simple_feature_calculator import SimpleFeatureCalculator
from research.model_pick.candle_fetch import FusionCandles
from strategies.BinanceBtcDeapV1Voting.models.config import FEAT_FRACDIFF, ALL_RAW_FEAT, SSMContainer
from jesse.helpers import date_to_timestamp
import lightgbm as lgb
import pandas as pd
import numpy as np

candle_container = FusionCandles(
    exchange="Binance Perpetual Futures", symbol="BTC-USDT", timeframe="1m"
)
candles = candle_container.get_candles("2022-07-01", "2025-10-01")

feat_calc = SimpleFeatureCalculator()
feat_calc.load(candles, sequential=True)
print("getting raw features...")
df_raw_feats = pd.DataFrame.from_dict(feat_calc.get(ALL_RAW_FEAT))
df_raw_feats.set_index(candles[:, 0].astype(int), inplace=True)
print(f"{df_raw_feats.shape = }")

model_deep_ssm = SSMContainer(model_type="deep_ssm")
model_lg_ssm = SSMContainer(model_type="lg_ssm")

df_deep_ssm = model_deep_ssm.transform(df_raw_feats[FEAT_FRACDIFF])
df_lg_ssm = model_lg_ssm.transform(df_raw_feats[FEAT_FRACDIFF])
df_feat_full = pd.concat([df_deep_ssm, df_lg_ssm, df_raw_feats], axis=1)

test_features = df_feat_full[
    (df_feat_full.index.to_numpy() >= date_to_timestamp("2025-03-01")) &
    (df_feat_full.index.to_numpy() < date_to_timestamp("2025-09-30"))
]

PyTorch configured: device=cpu, dtype=torch.float32


/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.12/site-packages/jesse/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


getting raw features...
df_raw_feats.shape = (36656, 12872)
Model loaded from /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/deep_ssm.safetensors and /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/deep_ssm.json
Model loaded from /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/lg_ssm.safetensors and /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/lg_ssm.json


In [4]:
import json

MODEL_NAME = "c_L5_N2"

with open("strategies/BinanceBtcDeapV1Voting/models/feature_info.json", "r") as f:
    feat_info = json.load(f)
df_selected_feats = test_features[feat_info[f"{MODEL_NAME}"]]

timestamps = candles[:, 0].astype(np.int64)
mask = (timestamps >= test_features.index[0]) & (timestamps <= test_features.index[-1])

volume = candles[:, 5][mask]
datetimes = pd.to_datetime(test_features.index, unit="ms")

assert len(volume) == len(datetimes)

model = lgb.Booster(model_file=f"strategies/BinanceBtcDeapV1Voting/models/model_{MODEL_NAME}.txt")
res = model.predict(df_selected_feats)
assert len(res) == len(datetimes)

In [5]:
from research.plot import analyze_confidence_slices

analyze_confidence_slices(
    datetimes,
    res,
    volume,
    lower_bound=0,
    upper_bound=1,
    threshold=0.5,
)

已生成: slice_0.9900_1.0000_ratio_0.0000.jpg (切片区间: [0.9900, 1.0000] (做多), 样本占比: 0.00%)
已生成: slice_0.9800_0.9900_ratio_0.0004.jpg (切片区间: [0.9800, 0.9900] (做多), 样本占比: 0.04%)
已生成: slice_0.9700_0.9800_ratio_0.0077.jpg (切片区间: [0.9700, 0.9800] (做多), 样本占比: 0.77%)
已生成: slice_0.9600_0.9700_ratio_0.0181.jpg (切片区间: [0.9600, 0.9700] (做多), 样本占比: 1.81%)
已生成: slice_0.9500_0.9600_ratio_0.0201.jpg (切片区间: [0.9500, 0.9600] (做多), 样本占比: 2.01%)
已生成: slice_0.9400_0.9500_ratio_0.0202.jpg (切片区间: [0.9400, 0.9500] (做多), 样本占比: 2.02%)
已生成: slice_0.9300_0.9400_ratio_0.0187.jpg (切片区间: [0.9300, 0.9400] (做多), 样本占比: 1.87%)
已生成: slice_0.9200_0.9300_ratio_0.0175.jpg (切片区间: [0.9200, 0.9300] (做多), 样本占比: 1.75%)
已生成: slice_0.9100_0.9200_ratio_0.0183.jpg (切片区间: [0.9100, 0.9200] (做多), 样本占比: 1.83%)
已生成: slice_0.9000_0.9100_ratio_0.0193.jpg (切片区间: [0.9000, 0.9100] (做多), 样本占比: 1.93%)
已生成: slice_0.8900_0.9000_ratio_0.0174.jpg (切片区间: [0.8900, 0.9000] (做多), 样本占比: 1.74%)
已生成: slice_0.8800_0.8900_ratio_0.0141.jpg (切片区间: [0.8800, 0.8900]

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

data_1 = pd.read_csv('model_l4_scores.csv')          ############   测试文件

data_1_size = data_1.shape[0]     ###### 测试数据行数  ###############
m_size = 25     ####### 测试多少个月 #######
buy = 1     ##### 多 ###################
sell = 0     ##### 空 ####################
rrr = 0.25     ###### 系数 ###################
m = 1000     ###### 总资金 ###################

In [2]:
ft = 0.01   ### 0.1, 0.05, 0.01

if ft == 0.1:
    da = 1
    xiao = 0.9
    z = 5

if ft == 0.05:
    da = 1
    xiao = 0.95
    z = 10
    
if ft == 0.01:
    da = 1
    xiao = 0.99
    z = 50


for i in range(z):        

    data_1['final'] = 0

    mask = (data_1['score'] < da) & (data_1['score'] >= xiao)
    data_1['final'] = np.where(mask, data_1['volume'], 0)
    
    true_count = mask.sum() / data_1_size 
    
    ### 更新置信度
    da = round((da - ft), 2)
    xiao = round((xiao - ft), 2)

    data_1['high'] = data_1['final'].cumsum()
    data_1['open'] = rrr * data_1['high'] / m

    fig=plt.figure(figsize=(20,10))  
    plt.plot(data_1['eob'], data_1['open'])
    plt.xticks(range(0,data_1_size,int(data_1_size/m_size)))   
    fig.autofmt_xdate()
    plt.grid(1)
    plt.savefig("./temp/" + str(da) + "_" + str(true_count.round(2)) + ".jpg")   
    plt.close()